# Soil Moisture Bin 1 Stability Experiments

**Objective**: Make soil_bin = 1 (SM ∈ [0.333, 0.667)) dynamically STABLE through physical dynamics changes.

**Goal**: Mean residence time in bin 1 ≥ 10 consecutive steps under random policy.

**Constraints**:
- NO drainage/runoff/percolation
- NO reward function changes
- NO discretization changes
- NO Q-learning changes

**Allowed modifications**:
- Climate sampling (rain/ET distributions)
- Water balance scaling
- Soil capacity parameters
- ET formulation

## Setup

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

sys.path.insert(0, '.')
sys.path.insert(0, '..')

from irrigation_env import IrrigationEnv
from irr_Qtable import discretize_state, N_ACTIONS

print("Modules imported successfully")

Modules imported successfully


## Step 1: Stability Instrumentation

Track residence time in soil_bin = 1

In [2]:
def measure_bin1_stability(env, n_episodes=100, n_soil_bins=3, verbose=False):
    """
    Measure residence time in soil_bin = 1 under random policy.
    
    Parameters
    ----------
    env : IrrigationEnv
        Environment instance
    n_episodes : int
        Number of episodes to run
    n_soil_bins : int
        Number of soil bins (default 3 for bins 0, 1, 2)
    verbose : bool
        Print detailed diagnostics
    
    Returns
    -------
    stats : dict
        Dictionary with stability statistics
    """
    # Storage for residence times
    residence_times = []  # Each entry is consecutive steps in bin 1
    bin_visits = defaultdict(int)  # Count visits to each bin
    entry_count = 0  # Number of times we enter bin 1
    
    # Track bin sequences for analysis
    all_bin_sequences = []
    
    for episode in range(n_episodes):
        obs, _ = env.reset()
        done = False
        
        # Track current stay in bin 1
        in_bin1 = False
        current_residence = 0
        bin_sequence = []
        
        while not done:
            # Get current soil bin
            state_idx = discretize_state(obs, n_soil_bins)
            soil_bin, crop_stage, et0_bin, rain_bin = from_discrete_to_components(state_idx, n_soil_bins)
            
            bin_sequence.append(soil_bin)
            bin_visits[soil_bin] += 1
            
            # Track entry and exit from bin 1
            if soil_bin == 1:
                if not in_bin1:
                    # Just entered bin 1
                    in_bin1 = True
                    entry_count += 1
                    current_residence = 1
                else:
                    # Still in bin 1
                    current_residence += 1
            else:
                if in_bin1:
                    # Just exited bin 1, record residence time
                    residence_times.append(current_residence)
                    in_bin1 = False
                    current_residence = 0
            
            # Random action (pure exploration)
            action = np.random.randint(N_ACTIONS)
            obs, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
        
        # If episode ended while in bin 1, record the residence time
        if in_bin1:
            residence_times.append(current_residence)
        
        all_bin_sequences.append(bin_sequence)
    
    # Calculate statistics
    if len(residence_times) > 0:
        mean_residence = np.mean(residence_times)
        median_residence = np.median(residence_times)
        max_residence = np.max(residence_times)
        min_residence = np.min(residence_times)
    else:
        mean_residence = 0
        median_residence = 0
        max_residence = 0
        min_residence = 0
    
    total_steps = sum(bin_visits.values())
    bin1_percentage = 100 * bin_visits[1] / total_steps if total_steps > 0 else 0
    
    stats = {
        'residence_times': residence_times,
        'mean_residence': mean_residence,
        'median_residence': median_residence,
        'max_residence': max_residence,
        'min_residence': min_residence,
        'entry_count': entry_count,
        'bin_visits': dict(bin_visits),
        'bin1_percentage': bin1_percentage,
        'total_steps': total_steps,
        'n_episodes': n_episodes,
    }
    
    if verbose:
        print(f"\n{'='*60}")
        print(f"BIN 1 STABILITY REPORT ({n_episodes} episodes, random policy)")
        print(f"{'='*60}")
        print(f"Total steps: {total_steps}")
        print(f"Bin 1 entries: {entry_count}")
        print(f"Bin 1 time percentage: {bin1_percentage:.1f}%")
        print(f"\nResidence time in bin 1:")
        print(f"  Mean: {mean_residence:.2f} steps")
        print(f"  Median: {median_residence:.1f} steps")
        print(f"  Min: {min_residence} steps")
        print(f"  Max: {max_residence} steps")
        print(f"  Total residence measurements: {len(residence_times)}")
        print(f"\nBin visit distribution:")
        for bin_idx in sorted(bin_visits.keys()):
            pct = 100 * bin_visits[bin_idx] / total_steps
            print(f"  Bin {bin_idx}: {bin_visits[bin_idx]:5d} visits ({pct:5.1f}%)")
        print(f"{'='*60}\n")
    
    return stats


def from_discrete_to_components(state_idx, n_soil_bins):
    """Extract state components from discrete state index."""
    rain_bin = state_idx % 2
    et0_bin = (state_idx // 2) % 2
    crop_stage = (state_idx // (2 * 2)) % 3
    soil_bin = state_idx // (3 * 2 * 2)
    return (soil_bin, crop_stage, et0_bin, rain_bin)


def plot_residence_histogram(stats, title="Bin 1 Residence Time Distribution"):
    """Plot histogram of residence times."""
    residence_times = stats['residence_times']
    
    if len(residence_times) == 0:
        print("No entries into bin 1 - cannot plot histogram")
        return
    
    plt.figure(figsize=(10, 6))
    plt.hist(residence_times, bins=range(1, max(residence_times) + 2), 
             edgecolor='black', alpha=0.7)
    plt.axvline(stats['mean_residence'], color='red', linestyle='--', 
                linewidth=2, label=f"Mean: {stats['mean_residence']:.2f}")
    plt.axvline(stats['median_residence'], color='orange', linestyle='--', 
                linewidth=2, label=f"Median: {stats['median_residence']:.1f}")
    plt.axvline(10, color='green', linestyle='-', linewidth=2, 
                label='Target: 10 steps', alpha=0.5)
    
    plt.xlabel('Consecutive Steps in Bin 1', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.title(title, fontsize=14)
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

print("Stability measurement functions defined")

Stability measurement functions defined


## Step 2: Baseline Measurement

Measure current stability with default parameters

In [3]:
# Baseline configuration
print("BASELINE CONFIGURATION:")
print("  max_et0: 8.0")
print("  max_rain: 50.0")
print("  et0_range: (2.0, 8.0)")
print("  rain_range: (0.0, 40.0)")
print("  max_soil_moisture: 100.0")
print("  episode_length: 90")
print("  n_soil_bins: 3")
print("  Bin 1 range: [33.3, 66.7) mm or [0.333, 0.667) normalized\n")

env_baseline = IrrigationEnv(
    max_et0=8.0,
    max_rain=50.0,
    et0_range=(2.0, 8.0),
    rain_range=(0.0, 40.0),
    max_soil_moisture=100.0,
    episode_length=90,
)

baseline_stats = measure_bin1_stability(env_baseline, n_episodes=100, n_soil_bins=3, verbose=True)

BASELINE CONFIGURATION:
  max_et0: 8.0
  max_rain: 50.0
  et0_range: (2.0, 8.0)
  rain_range: (0.0, 40.0)
  max_soil_moisture: 100.0
  episode_length: 90
  n_soil_bins: 3
  Bin 1 range: [33.3, 66.7) mm or [0.333, 0.667) normalized


BIN 1 STABILITY REPORT (100 episodes, random policy)
Total steps: 9000
Bin 1 entries: 101
Bin 1 time percentage: 1.6%

Residence time in bin 1:
  Mean: 1.43 steps
  Median: 1.0 steps
  Min: 1 steps
  Max: 5 steps
  Total residence measurements: 101

Bin visit distribution:
  Bin 1:   144 visits (  1.6%)
  Bin 2:  8856 visits ( 98.4%)



In [ ]:
# Visualize baseline
plot_residence_histogram(baseline_stats, "BASELINE: Bin 1 Residence Time")

## Step 3: Controlled Dynamics Experiments

One-factor-at-a-time experiments to improve bin 1 stability

### Experiment A: Rain Process Weakening

In [4]:
# A1: Reduce rain upper bound
print("\n" + "="*70)
print("EXPERIMENT A1: Reduce rain_range from (0, 40) to (0, 20)")
print("="*70)

env_a1 = IrrigationEnv(
    max_et0=8.0,
    max_rain=50.0,
    et0_range=(2.0, 8.0),
    rain_range=(0.0, 20.0),  # Reduced from 40
    max_soil_moisture=100.0,
    episode_length=90,
)

stats_a1 = measure_bin1_stability(env_a1, n_episodes=100, n_soil_bins=3, verbose=True)


EXPERIMENT A1: Reduce rain_range from (0, 40) to (0, 20)

BIN 1 STABILITY REPORT (100 episodes, random policy)
Total steps: 9000
Bin 1 entries: 103
Bin 1 time percentage: 2.4%

Residence time in bin 1:
  Mean: 2.06 steps
  Median: 2.0 steps
  Min: 1 steps
  Max: 6 steps
  Total residence measurements: 103

Bin visit distribution:
  Bin 1:   212 visits (  2.4%)
  Bin 2:  8788 visits ( 97.6%)



In [5]:
# A2: Further reduce rain
print("\n" + "="*70)
print("EXPERIMENT A2: Reduce rain_range from (0, 20) to (0, 10)")
print("="*70)

env_a2 = IrrigationEnv(
    max_et0=8.0,
    max_rain=50.0,
    et0_range=(2.0, 8.0),
    rain_range=(0.0, 10.0),  # Further reduced
    max_soil_moisture=100.0,
    episode_length=90,
)

stats_a2 = measure_bin1_stability(env_a2, n_episodes=100, n_soil_bins=3, verbose=True)


EXPERIMENT A2: Reduce rain_range from (0, 20) to (0, 10)

BIN 1 STABILITY REPORT (100 episodes, random policy)
Total steps: 9000
Bin 1 entries: 109
Bin 1 time percentage: 3.5%

Residence time in bin 1:
  Mean: 2.85 steps
  Median: 2.0 steps
  Min: 1 steps
  Max: 9 steps
  Total residence measurements: 109

Bin visit distribution:
  Bin 1:   311 visits (  3.5%)
  Bin 2:  8689 visits ( 96.5%)



### Experiment B: Increase Soil Water Capacity

In [6]:
# B1: Increase field capacity to 150 mm
print("\n" + "="*70)
print("EXPERIMENT B1: Increase max_soil_moisture from 100 to 150 mm")
print("="*70)

env_b1 = IrrigationEnv(
    max_et0=8.0,
    max_rain=50.0,
    et0_range=(2.0, 8.0),
    rain_range=(0.0, 40.0),
    max_soil_moisture=150.0,  # Increased from 100
    episode_length=90,
)

stats_b1 = measure_bin1_stability(env_b1, n_episodes=100, n_soil_bins=3, verbose=True)


EXPERIMENT B1: Increase max_soil_moisture from 100 to 150 mm

BIN 1 STABILITY REPORT (100 episodes, random policy)
Total steps: 9000
Bin 1 entries: 100
Bin 1 time percentage: 2.3%

Residence time in bin 1:
  Mean: 2.06 steps
  Median: 2.0 steps
  Min: 1 steps
  Max: 11 steps
  Total residence measurements: 100

Bin visit distribution:
  Bin 1:   206 visits (  2.3%)
  Bin 2:  8794 visits ( 97.7%)



In [7]:
# B2: Further increase to 200 mm
print("\n" + "="*70)
print("EXPERIMENT B2: Increase max_soil_moisture from 150 to 200 mm")
print("="*70)

env_b2 = IrrigationEnv(
    max_et0=8.0,
    max_rain=50.0,
    et0_range=(2.0, 8.0),
    rain_range=(0.0, 40.0),
    max_soil_moisture=200.0,  # Further increased
    episode_length=90,
)

stats_b2 = measure_bin1_stability(env_b2, n_episodes=100, n_soil_bins=3, verbose=True)


EXPERIMENT B2: Increase max_soil_moisture from 150 to 200 mm

BIN 1 STABILITY REPORT (100 episodes, random policy)
Total steps: 9000
Bin 1 entries: 101
Bin 1 time percentage: 3.1%

Residence time in bin 1:
  Mean: 2.79 steps
  Median: 2.0 steps
  Min: 1 steps
  Max: 11 steps
  Total residence measurements: 101

Bin visit distribution:
  Bin 1:   282 visits (  3.1%)
  Bin 2:  8718 visits ( 96.9%)



### Experiment C: ET Strengthening

**Note**: ET strengthening requires modifying the irrigation_env.py file to change kc values.
This will be done after identifying the most promising approach from A and B experiments.

### Experiment D: Combined Approach

In [8]:
# D1: Combine reduced rain + increased capacity
print("\n" + "="*70)
print("EXPERIMENT D1: rain_range=(0,15) + max_soil_moisture=150")
print("="*70)

env_d1 = IrrigationEnv(
    max_et0=8.0,
    max_rain=50.0,
    et0_range=(2.0, 8.0),
    rain_range=(0.0, 15.0),      # Moderate reduction
    max_soil_moisture=150.0,     # Moderate increase
    episode_length=90,
)

stats_d1 = measure_bin1_stability(env_d1, n_episodes=100, n_soil_bins=3, verbose=True)


EXPERIMENT D1: rain_range=(0,15) + max_soil_moisture=150

BIN 1 STABILITY REPORT (100 episodes, random policy)
Total steps: 9000
Bin 1 entries: 102
Bin 1 time percentage: 3.9%

Residence time in bin 1:
  Mean: 3.44 steps
  Median: 3.0 steps
  Min: 1 steps
  Max: 13 steps
  Total residence measurements: 102

Bin visit distribution:
  Bin 1:   351 visits (  3.9%)
  Bin 2:  8649 visits ( 96.1%)



In [ ]:
# D2: More aggressive: rain↓5 + capacity↑200
print("\n" + "="*70)
print("EXPERIMENT D2: rain_range=(0,5) + max_soil_moisture=200")
print("="*70)

env_d2 = IrrigationEnv(
    max_et0=8.0,
    max_rain=50.0,
    et0_range=(2.0, 8.0),
    rain_range=(0.0, 5.0),       # Very low rain
    max_soil_moisture=200.0,     # High capacity
    episode_length=90,
)

stats_d2 = measure_bin1_stability(env_d2, n_episodes=100, n_soil_bins=3, verbose=True)

## Step 4: Results Comparison

In [9]:
# Compile all results
experiments = {
    'Baseline': baseline_stats,
    'A1 (rain↓20)': stats_a1,
    'A2 (rain↓10)': stats_a2,
    'B1 (capacity↑150)': stats_b1,
    'B2 (capacity↑200)': stats_b2,
    'D1 (rain↓15+cap↑150)': stats_d1,
}

print("\n" + "="*80)
print("STABILITY COMPARISON: Mean Residence Time in Bin 1")
print("="*80)
print(f"{'Experiment':<25} {'Mean (steps)':<15} {'Median (steps)':<15} {'Entries':<10} {'Status'}")
print("-"*80)

for name, stats in experiments.items():
    mean_res = stats['mean_residence']
    median_res = stats['median_residence']
    entries = stats['entry_count']
    status = "✓ STABLE" if mean_res >= 10 else "✗ Unstable"
    
    print(f"{name:<25} {mean_res:<15.2f} {median_res:<15.1f} {entries:<10d} {status}")

print("="*80)
print("\nTarget: Mean residence time ≥ 10 consecutive steps")
print("="*80)


STABILITY COMPARISON: Mean Residence Time in Bin 1
Experiment                Mean (steps)    Median (steps)  Entries    Status
--------------------------------------------------------------------------------
Baseline                  1.43            1.0             101        ✗ Unstable
A1 (rain↓20)              2.06            2.0             103        ✗ Unstable
A2 (rain↓10)              2.85            2.0             109        ✗ Unstable
B1 (capacity↑150)         2.06            2.0             100        ✗ Unstable
B2 (capacity↑200)         2.79            2.0             101        ✗ Unstable
D1 (rain↓15+cap↑150)      3.44            3.0             102        ✗ Unstable

Target: Mean residence time ≥ 10 consecutive steps


In [ ]:
# Visual comparison
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (name, stats) in enumerate(experiments.items()):
    if stats['residence_times']:
        ax = axes[idx]
        ax.hist(stats['residence_times'], bins=range(1, max(stats['residence_times']) + 2),
                edgecolor='black', alpha=0.7)
        ax.axvline(stats['mean_residence'], color='red', linestyle='--', linewidth=2)
        ax.axvline(10, color='green', linestyle='-', linewidth=2, alpha=0.5)
        ax.set_xlabel('Steps in Bin 1')
        ax.set_ylabel('Frequency')
        ax.set_title(f"{name}\nMean: {stats['mean_residence']:.2f}")
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 5: Final Report

In [ ]:
print("\n" + "#"*80)
print("#" + " "*78 + "#")
print("#" + "  FINAL STABILITY REPORT: Soil Bin 1 Dynamics".center(78) + "#")
print("#" + " "*78 + "#")
print("#"*80)

# Find best configuration
best_name = max(experiments.keys(), key=lambda k: experiments[k]['mean_residence'])
best_stats = experiments[best_name]

print(f"\n{'='*80}")
print("BASELINE RESULTS")
print(f"{'='*80}")
print(f"Mean residence time: {baseline_stats['mean_residence']:.2f} steps")
print(f"Median residence time: {baseline_stats['median_residence']:.1f} steps")
print(f"Bin 1 entries: {baseline_stats['entry_count']}")
print(f"Status: {'UNSTABLE' if baseline_stats['mean_residence'] < 10 else 'STABLE'}")

print(f"\n{'='*80}")
print("BEST CONFIGURATION")
print(f"{'='*80}")
print(f"Experiment: {best_name}")
print(f"Mean residence time: {best_stats['mean_residence']:.2f} steps")
print(f"Median residence time: {best_stats['median_residence']:.1f} steps")
print(f"Bin 1 entries: {best_stats['entry_count']}")
print(f"Status: {'STABLE ✓' if best_stats['mean_residence'] >= 10 else 'UNSTABLE ✗'}")

improvement = best_stats['mean_residence'] - baseline_stats['mean_residence']
print(f"\nImprovement: {improvement:+.2f} steps ({improvement/baseline_stats['mean_residence']*100:+.1f}%)")

print(f"\n{'='*80}")
print("KEY FINDINGS")
print(f"{'='*80}")

# Analyze which factor had most impact
rain_effect = stats_a2['mean_residence'] - baseline_stats['mean_residence']
capacity_effect = stats_b2['mean_residence'] - baseline_stats['mean_residence']

print(f"\n1. Rain reduction impact: {rain_effect:+.2f} steps")
print(f"   (rain_range 40→10 mm/day)")

print(f"\n2. Soil capacity impact: {capacity_effect:+.2f} steps")
print(f"   (max_soil_moisture 100→200 mm)")

if abs(rain_effect) > abs(capacity_effect):
    print(f"\n   → Rain modification had LARGER effect")
else:
    print(f"\n   → Soil capacity had LARGER effect")

print(f"\n3. Combined approach (D1) mean residence: {stats_d1['mean_residence']:.2f} steps")

print(f"\n{'='*80}")
print("CONCLUSION")
print(f"{'='*80}")

if best_stats['mean_residence'] >= 10:
    print("\n✓ SUCCESS: Bin 1 is now DYNAMICALLY STABLE")
    print(f"  Mean residence time of {best_stats['mean_residence']:.2f} steps meets target (≥10)")
else:
    print("\n✗ Target not yet achieved")
    print(f"  Current best: {best_stats['mean_residence']:.2f} steps")
    print(f"  Gap to target: {10 - best_stats['mean_residence']:.2f} steps")
    print("\n  Recommendations:")
    print("  - Consider ET strengthening (modify kc values)")
    print("  - Try more aggressive parameter combinations")
    print("  - Consider moisture-dependent ET formulation")

print(f"\n{'#'*80}\n")